In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [ ]:
import getpass
import os
from langchain.globals import set_debug

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

··········


In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
set_debug(True)


In [ ]:
# Load, chunk and index the contents of the blog.
txt = """
  Welcome t
"""

urls = "https://ics.uci.edu/~thornton/ics33/Notes/CourseIntroduction,https://ics.uci.edu/~thornton/ics33/Notes/ModulesAndNamespaces,https://ics.uci.edu/~thornton/ics33/Notes/ClassesAndObjects,https://ics.uci.edu/~thornton/ics33/Notes/Parameters,https://ics.uci.edu/~thornton/ics33/Notes/ContextManagers,https://ics.uci.edu/~thornton/ics33/Notes/AsymptoticAnalysis,https://ics.uci.edu/~thornton/ics33/Notes/Searching,https://ics.uci.edu/~thornton/ics33/Notes/Databases,https://ics.uci.edu/~thornton/ics33/Notes/Comprehensions,https://ics.uci.edu/~thornton/ics33/Notes/Iteration,https://ics.uci.edu/~thornton/ics33/Notes/Generators,https://ics.uci.edu/~thornton/ics33/Notes/UsingGenerators,https://ics.uci.edu/~thornton/ics33/Notes/PythonDataModel,https://ics.uci.edu/~thornton/ics33/Notes/Inheritance,https://ics.uci.edu/~thornton/ics33/Notes/Recursion,https://ics.uci.edu/~thornton/ics33/Notes/FunctionalProgramming,https://ics.uci.edu/~thornton/ics33/Notes/Decorators,https://ics.uci.edu/~thornton/ics33/Notes/ClassDesign,https://ics.uci.edu/~thornton/ics33/Notes/AbstractBaseClasses"
urls = urls.split(",")

loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("section")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What are the programming background requirements for ICS 33?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the programming background requirements for ICS 33?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What are the programming background requirements for ICS 33?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the programming background requirements for ICS 33?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What are the programming background requirements for ICS 33?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What are the programming background requireme

'The programming background requirements for ICS 33 include a Python background equivalent to completing ICS 31 and ICS 32 courses or ICS 32A course. The Notes and Examples from the most recent ICS 32A offering provide a complete overview of the Python knowledge expected at the beginning of the course. If the majority of the listed topics are new to you, you may find the course challenging.'

In [ ]:
rag_chain.invoke("What are we reccomended to call keyword arguments? Where are they to be placed")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are we reccomended to call keyword arguments? Where are they to be placed"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What are we reccomended to call keyword arguments? Where are they to be placed"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are we reccomended to call keyword arguments? Where are they to be placed"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What are we reccomended to call keyword arguments? Where are they to be placed"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] s] Exiting Chain run w

'Keyword arguments are recommended to be called keyword-only parameters. They are to be placed after positional arguments in Python.'